### Data Reading

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@rtgpadlsgen2.dfs.core.windows.net/orders")
display(df)

In [0]:
df.printSchema()


In [0]:
df = df.withColumnRenamed('_rescued_data','rescued_data')

In [0]:
df = df.drop('rescued_data')
df.display()

In [0]:
from pyspark.sql.functions import to_timestamp, col

df = df.withColumn("order_date", to_timestamp(col("order_date")))
df.display()

In [0]:
from pyspark.sql.functions import year, col

df = df.withColumn("year", year(col('order_date')))
display(df)

In [0]:
from pyspark.sql.functions import dense_rank, col
from pyspark.sql.window import Window

df1 = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
display(df1)



In [0]:
from pyspark.sql.functions import col, rank
from pyspark.sql.window import Window

df2 = df.withColumn("flag",rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))

display(df2)

In [0]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

df3 = df.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))

display(df3)

### Classes - OOPs

In [0]:
class windows:
    def dense_rank(self,df):
      df_dense_rank = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
      return df_dense_rank
    
    def rank(self,df):
      df_rank = df.withColumn("flag",rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
      return df_rank
    
    def row_number(self,df):
      df_row_number = df.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
      return df_row_number
  



In [0]:
df_new = df


In [0]:
df_new.display()

In [0]:
obj = windows()

In [0]:
df_result= obj.dense_rank(df_new)
df_result.display()

DATA WRITING


In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@rtgpadlsgen2.dfs.core.windows.net/orders")

In [0]:
%sql
create table if not exists databricks_cata.silver.orders
using delta
location 'abfss://silver@rtgpadlsgen2.dfs.core.windows.net/orders'

In [0]:
%sql
select * from databricks_cata.silver.orders